## 图像质量评估指标公式与参数解释


### 关键特性对比
| 指标   | 评估对象       | 输入类型       | 值域方向      | 典型应用场景       |
|--------|----------------|----------------|---------------|--------------------|
| SSIM   | 单对图像       | 像素级对比     | 值越高越好    | 图像修复/超分辨率 |
| FID    | 数据分布       | 特征空间分布   | 值越低越好    | GAN模型评估       |

---

### 1. SSIM (结构相似性指数)

#### 公式
$$
\text{SSIM}(x, y) = \frac{(2\mu_x \mu_y + C_1)(2\sigma_{xy} + C_2)}{(\mu_x^2 + \mu_y^2 + C_1)(\sigma_x^2 + \sigma_y^2 + C_2)}
$$

#### 参数说明
- $\mu_x, \mu_y$: 图像 $x$ 和 $y$ 的**局部均值**  
- $\sigma_x^2, \sigma_y^2$: 图像 $x$ 和 $y$ 的**局部方差**  
- $\sigma_{xy}$: 图像 $x$ 和 $y$ 之间的**协方差**  
- $C_1, C_2$: 防止分母为零的**稳定常数**，通常定义为：
  - $C_1 = (k_1 L)^2$,  $C_2 = (k_2 L)^2$  
  - 其中 $L$ 是像素值的动态范围（如 `255` 对应 8-bit 图像）  
  - $k_1 = 0.01$, $k_2 = 0.03$（默认值）

---

### 2. FID (Frechet Inception Distance)

#### 公式
$$
\text{FID} = \|\mu_P - \mu_Q\|^2 + \text{Tr}\left( \Sigma_P + \Sigma_Q - 2 (\Sigma_P \Sigma_Q)^{1/2} \right)
$$

#### 参数说明
- $\mu_P, \mu_Q$: 真实数据分布 $P$ 和生成数据分布 $Q$ 的**特征均值向量**  
  - 通过预训练模型（如 Inception-v3）提取的特征向量的均值
- $\Sigma_P, \Sigma_Q$: 真实数据分布 $P$ 和生成数据分布 $Q$ 的**协方差矩阵**  
  - 描述特征向量之间的相关性
- $\text{Tr}(\cdot)$: 矩阵的**迹**（对角线元素之和）  
- $(\Sigma_P \Sigma_Q)^{1/2}$: 协方差矩阵乘积的**平方根矩阵**  
  - 需通过数值方法（如 SVD）计算实数解

In [ ]:
from torchvision import transforms
from model.Normalization import *

transform=transforms.Compose([
    # transforms.Normalize((0,),(10.0,))
    # TanhNorm(scale=25)
    # MinMaxNorm(0,10)
])
# transform=None

In [ ]:
from model.RFID_Dataset import RFID_Dataset

# 训练集
train_dataset=RFID_Dataset(
    r"data\RFID\dataset\all",
    T=32,
    step=1,
    transform=transform,
)
print(f"训练集的数据个数: {len(train_dataset)}")

In [ ]:
from model.RFID_Dataset import RFID_Dataset

# 测试集
test_dataset=RFID_Dataset(
    r"./output/improve_v1",
    T=32,
    step=1,
    transform=transform,
)
print(f"测试集的数据个数: {len(test_dataset)}")

In [ ]:
from utils.SimilarityUtils import *

used_dataset=train_dataset

all_datas=used_dataset[:][0]

print(f"mean: {all_datas.mean()}")
print(f"std: {all_datas.std()}")
print(f"feature_mean: {all_datas.mean(dim=(0,1,2))}")
print(f"feature_std: {all_datas.std(dim=(0,1,2))}")

for i in range(3):

    datas = filter_datas(used_dataset, i)
    print(f"类别{i}的数据个数: {len(datas)}")
    print(f"mean: {datas.mean()}")
    print(f"std: {datas.std()}")
    print("=" * 30)

In [ ]:
from utils.SimilarityUtils import *

num_classes =3
for i in range(num_classes):
    train_datas=filter_datas(train_dataset, i)
    fid_train_dataset=FID_Dataset(
        train_datas
    )

    test_datas=filter_datas(test_dataset, i)
    fid_test_dataset=FID_Dataset(
        test_datas,
    )

    print(f"Condition: {i}")
    print(f"训练集的数据个数: {len(fid_train_dataset)}")
    print(f"测试集的数据个数: {len(fid_test_dataset)}")

    # 提取特征值
    train_features=extract_inception_features(fid_train_dataset)
    test_features=extract_inception_features(fid_test_dataset)

    # 计算FID分数
    fid_score=calculate_fid(train_features, test_features)
    print(f"FID: {fid_score:.6f}")
    print("="*30)